In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

def generate(instruction, knowledge, dialog):
    if knowledge != '':
        knowledge = '[KNOWLEDGE] ' + knowledge
    dialog = ' EOS '.join(dialog)
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    input_ids = tokenizer(f"{query}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128, min_length=8, top_p=0.9, do_sample=True)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, you need to response empathically.'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Does money buy happiness?',
    'It is a question. Money buys you a lot of things, but not enough to buy happiness.',
    'What is the best way to buy happiness ?'
]
response = generate(instruction, knowledge, dialog)
print(response)

I know that you are not very knowledgeable, but some people think that money can buy happiness.


In [2]:
# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, you need answer factual questions about movies'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Do you know by any chance what were the cast members of this movie with Keanu Reeves and Sandra Bullock, you know, The Lake House',
    #'Sorry, could you phrase the question more simply?',
    'Who are the cast members of the lake house besides Keanu Reeves and Sandra Bullock?',
    #"Sorry, I think you are referring to many things at the same time (it's harder for me to understand :)). Could you make the question simpler?",
    "Who are the cast members of the Lake House?"
]

response = generate(instruction, knowledge, dialog)
print(response)

Yes, Keanu Reeves, Sandra Bullock and The Lake House.


### Small talk and redirect to talking about movies

In [9]:
instruction = f'Instruction: given a dialog context, respond basic questions or small talk and always change the conversation to movies or films'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'hi',
    'Hey! how is it going?',
    'All good, thanks, can you tell me a story?',
    'I really rather talk about movies',
    'helloooo, how are you?'
]

response = generate(instruction, knowledge, dialog)
print(response)

I'm good. how about you


This seems to work more or less well for small talk

### Attempt to answer factual questions

In [71]:
instruction = f'Instruction: given a dialog context, answer correctly questions about movies or films using information from imdb, wikidata, and wikipedia'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Who directed The Bridge on the River Kwai?',
    'David Lean directed The Bridge on the River Kwai'
    'Who is the director of Star Wars: Episode VI - Return of the Jedi?',
    'I think it is Richard Marquand',
    'Who is the screenwriter of The Masked Gang: Cyprus?',
    'The screenwriter of The Masked Gang: Cyprus is Murat Aslan'
    'What is the MPAA film rating of Weathering with you?',
    'The MPAA film rating of Weathering with you is PG-13'
    'What is the genre of Good Neighbors?',
    'The genre of Good Neighbors is neo noir',
    'What is the box office of Princess and the Frog?',
    'The box offic of The Princess and the Frog is 267000000'
    'Can you tell me the publication date of Tom Meets Zizou?',
    'The publication date of Tom Meets Zizou is 2011-01-01'
    'Who is the executive producer of X-Men: First Class?',
    'The executive producer is Sheryl Lee Ralph',


]

response = generate(instruction, knowledge, dialog)
print(response)

The genre of Friday the 13th is neo noir


In [171]:
instruction = f'Instruction: given a dialog context and knowledge from imdb and wikipedia, answer the question as accurately as possible'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Who directed The Bridge on the River Kwai?',
    'David Lean directed The Bridge on the River Kwai'
    'Who is the screenwriter of The Masked Gang: Cyprus?',
    'The screenwriter of The Masked Gang: Cyprus is Murat Aslan'
    'What is the MPAA film rating of Weathering with you?',
    'The MPAA film rating of Weathering with you is PG-13'
    'What is the genre of Good Neighbors?',
    'The genre of Good Neighbors is neo noir',
    'What is the box office of Princess and the Frog?',
    'The box offic of The Princess and the Frog is 267000000'
    'Can you tell me the publication date of Tom Meets Zizou?',
    'The publication date of Tom Meets Zizou is 2011-01-01'
    'Who is the executive producer of X-Men: First Class?',
    'The executive producer is Sheryl Lee Ralph',

    'What is the cast of the movie Amelie?'

]

response = generate(instruction, knowledge, dialog)
print(response)

The cast of Amelie (or the cast of Amelie in this film).


Less than ideal but at least it gives it a shot

### Attempt to make a set of movie recommendations

In [137]:
instruction = f'Instruction: given a dialog context, recommend movies that share some characteristics with the moci'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Recommend movies similar to Hamlet and Othello',
    'Adequate recommendations would be movies in the drama genre that are based on clasic literature (eg. Shakespeare, Dickens, or Jane Austen)',
    'Recommend movies similar to Hamlet and Othello',
    'I think adequate recommendations would be films in one of the following genre: drama; filmed in United Kingdom; that are based on play. I would suggest then trying Romeo and Juliet, The Tempest, Henry V, and Sense and Sensibility',
    'Given that I like The Lion King, Pocahontas, and The Beauty and the Beast, can you recommend some movies?',
    'I would recommend (2-D) animated movies or real-life remakes of Disney Movies',
    'Recommend movies like Nightmare on Elm Street, Friday the 13th and Halloween',
    'I recommend horror movies from the 1970s or 1980s or sequels to the movies',

    'Recommend movies like Raging Bull, Good Fellas, and Donny Brasco'


]

response = generate(instruction, knowledge, dialog)
print(response)

Raging Bull, Good Fellas, and Donny Brasco all have similar qualities. Do you like this movie?


In [140]:
instruction = f'Instruction: given a dialog context, recommend movies other than those already mentioned'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Recommend movies similar to Hamlet and Othello'

]

response = generate(instruction, knowledge, dialog)
print(response)

Yes, I think Hamlet was the best movie to watch. I also loved Othello. Do you like Hamlet?


Less than ideal but at least it gives it a shot

### Let's give it a shot with Flan T-5

In [141]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")

outputs = model.generate(**inputs)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
['Pour a cup of bolognese into a large bowl and add the pasta']

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

['Pour a cup of bolognese into a large bowl and add the pasta']


['Pour a cup of bolognese into a large bowl and add the pasta']

In [163]:
inputs = tokenizer("Please answer the next question as accurately as possible: What is the box office of Princess and the Frog?", return_tensors="pt")

outputs = model.generate(**inputs)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['£1 million']


Seems we cannot prompt it for that.